# Raw satellite products processing
A small notebook to process the satellite products we downloaded from various platforms.  
The processing varies depending on the mission we are considering.

## Libraries import

In [ ]:
import rasterio
import numpy as np
import os
import ntpath
import glob
import matplotlib.pyplot as plt
import pandas as pd
import sys
sys.path.append('..')
from isplutils.PatchExtractor import PatchExtractor
from matplotlib import rc
rc('font',**{'family':'sans-serif','sans-serif':['Times']})
## for Palatino and other serif fonts use:
#rc('font',**{'family':'serif','serif':['Palatino']})
rc('text', usetex=True)

## Helpers functions

In [ ]:
"""
Some image functions we are trying for normalizing 16-bit GRD images and convert them to 8-bits images.
For each scaling we provide the 8-bit conversion process too.

Authors:
Edoardo Daniele Cannas - edoardodaniele.cannas@polimi.it
"""
from sklearn.preprocessing import RobustScaler, QuantileTransformer


## Normalization functions

def raw_scaling(img: np.ndarray, conversion_8: bool = False) -> np.ndarray:
    """
    Standard min-max scaler

    :param img: np.ndarray, input image (tested GRD 16-bits)
    :param conversion_8: bool, wheter to convert to 8-bits depth
    :return: np.ndarray, normalized numpy array
    """
    norm = ((img-img.min())/(img.max()-img.min()))
    if conversion_8: # here
        norm = (norm*255).astype(np.uint8)
    return norm


def db_scaling(img: np.ndarray, conversion_8: bool = False) -> np.ndarray:
	"""
	Puts the image in logarithmic scale with a factor 10 
	
	:param img: np.ndarray, input image (tested GRD 16-bits)
	:param conversion_8: bool, wheter to convert to 8-bits depth
	:return: np.ndarray, normalized numpy array
	"""
	norm = np.log10(img)
	if conversion_8:
		norm = (norm-norm.min())/(norm.max()-norm.min())  # np.log10 provides the image already in float32, however the range may not be restricted 0-1
		norm = (norm*255).astype(np.uint8)
	return norm


def mul_scaling(img: np.ndarray,  mul_factor: int = 100, conversion_8: bool = False) -> np.ndarray:
	"""
	Simple scaling of the dynamic with a multiplicative factor
	
	:param img: np.ndarray, input image (tested GRD 16-bits)
	:param mul_factor: int, multiplicative factor for the scaling
	:param conversion_8: bool, wheter to convert to 8-bits depth
	:return: np.ndarray, normalized numpy array
	"""
	norm = ((img-img.min())/(img.max()-img.min()))
	norm *= mul_factor
	if conversion_8:
		norm = (norm*255).astype(np.uint8)
	return norm


def janos_scaling(img: np.ndarray, conversion_8: bool = False) -> np.ndarray:
	"""
	Nice dynamic scaling using exponential functions, original author Janos Horvath - horvath5@purdue.edu
	
	:param img: np.ndarray, input image (tested GRD 16-bits)
	:param conversion_8: bool, wheter to convert to 8-bits depth
	:return: np.ndarray, normalized numpy array
	"""
	norm = 2/((1+np.exp(-(img*40/256/256))))-1
	if conversion_8:
		norm = (norm*255).astype(np.uint8)
	return norm


def mz_scaling(img: np.ndarray, conversion_8: bool = False) -> np.ndarray:
    """
    Modified Z-score normalization taken from Sun et al, “DeepInSAR—A Deep Learning Framework for SAR Interferometric Phase Restoration and Coherence Estimation”

    :param img: np.ndarray, input image (tested GRD 16-bits)
    :param conversion_8: bool, wheter to convert to 8-bits depth
    :return: np.ndarray, normalized numpy array
    """
    shape = img.shape
    img = img.flatten()
    img_med = np.median(img)
    mad = np.median(np.abs(img - np.median(img)))
    #mad=0.1656
    mz = 0.6745 * ((img - np.median(img)) / mad)  # 0.6475 is the 75th quartile of the normal distribution, removes the outlier
    #mz = 0.6745 * ((a - 0.7097678) / mad)
    mz = (np.tanh(mz / 7) + 1) / 2
    mz_min = mz.min()
    mz_max = mz.max()
    mz = (mz - mz.min()) / (mz.max() - mz.min())
    norm = mz.reshape(shape)
    if conversion_8:
        norm = (norm*255).astype(np.uint8)
    return norm


## Range equalizers

def robust_scaling(img: np.ndarray) -> np.ndarray:
	"""
	RobustScaler: eliminates outliers, reproject the images in the same range
	
	:param img: np.ndarray, input image (tested GRD 16-bits)
	:return: np.ndarray, normalized numpy array
	"""
	return RobustScaler().fit_transform(img.reshape(-1, 1)).reshape(img.shape)
	

def quantile_scaling(img: np.ndarray, dist: str) -> np.ndarray:
	"""
	QuantileTransformer: maps the image range to a specific stastical distribution
	
	:param img: np.ndarray, input image (tested GRD 16-bits)
	:param distribution: str, distribution for mapping the image (either 'uniform' or 'normal')
	:return: np.ndarray, normalized numpy array
	"""
	return QuantileTransformer(output_distribution=dist).fit_transform(img.reshape(-1, 1)).reshape(img.shape)

## CBERS sample
These are samples from a joint [Chinese-Brazilian Earth Resource Satellite program](https://en.wikipedia.org/wiki/CBERS).  
Images are downloaded as separate tiffs for each band, so we should take a look at them and eventually re-assemble them together.

In [ ]:
DATA_ROOT = '../data/pristine_images/full_res_products'
bands = glob.glob(os.path.join(DATA_ROOT, '**/*.tif*'), recursive=True)
bands = [band for band in bands if 'analytic_sr' not in band]
data_df = pd.concat([pd.concat({acq.split('/')[4]: pd.DataFrame(index=[acq])}, names=['Original_product', 'Bands path']) for acq in bands])
data_df

### Let's add the info we need from the profile

In [ ]:
# We can add some information in between too
for product in data_df.index.get_level_values(0).unique():
    # Select the product
    df = data_df.loc[product]
    fsize = 20
    for idx, band in enumerate(df.index.get_level_values(0).unique()):
        # Load the band
        with rasterio.open(band, 'r') as src:
            # Add info
            data_df.loc[(product, band), 'height'] = src.profile['height']
            data_df.loc[(product, band), 'width'] = src.profile['width']
            data_df.loc[(product, band), 'bit_depth'] = src.profile['dtype']

### Let's plot all these bands

In [ ]:
# We can add some information in between too
for product in data_df.index.get_level_values(0).unique():
    # Select the product
    df = data_df.loc[product]
    fsize = 20
    # Prepare the figure
    if len(df) > 3:
        fig, axs = plt.subplots(1, len(df), figsize=(16*len(df), 16))
        fsize = 35
    else:
        fig, axs = plt.subplots(1, len(df), figsize=(12*len(df), 12))
    for idx, band in enumerate(df.index.get_level_values(0).unique()):
        # Load the band
        with rasterio.open(band, 'r') as src:
            raster = src.read()
        axs[idx].imshow(np.squeeze(raster), cmap='gray')
        axs[idx].set_title(f"{band.split('/')[-1].split('.')[0]} raster,\ndtype {raster.dtype}, dimensions {raster.shape}", fontsize=fsize)
    fig.suptitle(f"{product} bands", fontsize=fsize)
    plt.show()

### Let's glue them together. 
Watch out for the different meaning of bands (look [here](https://www.eoportal.org/satellite-missions/cbers-3-4#sensor-complement-muxcam-panmux-irs-wfi-dcs)).  

**MUX sensor**:
1. B05: 0.45 - 0.52 µm (Blue band);
2. B06: 0.52 - 0.59 µm (Green band);
3. B07: 0.63 - 0.69 µm (Red band);
4. B08: 0.77 - 0.89 µm (NIR band).

**PAN sensor**:
1. B01: 0.51 - 0.85 µm (panchromatic band);
2. B02: 0.52 - 0.59 µm (Green band);
3. B03: 0.63 - 0.69 µm (Red band);
4. B04: 0.77 - 0.89 µm (NIR band).

**WFI sensor**:
1. B13: 0.45 - 0.52 µm (Blue band);
2. B14: 0.52 - 0.59 µm (Green band);
3. B15: 0.63 - 0.69 µm (Red band);
4. B16: 0.77 - 0.89 µm (NIR band).

**CCD sensor**:
1. B1 = 0,45 - 0,52 µm (Blue band);
2. B2 = 0,52 - 0,59 µm (Green band);
3. B3 = 0,63 - 0,69 µm (Red band);
4. B4 = 0,77 - 0,89 µm (NIR band).

Since we are dealing with RGB images, we are going to work only with the Red, Green, and Blue bands.  
We are also adding the mask in the plot for no-data values.

### Let's modify a little the DataFrame

In [ ]:
# Let's pick up the sensors first and bands first
data_df['sensor'] = ''
data_df['bands_num'] = ''
for i, r in data_df.iterrows():
    data_df.loc[i, 'sensor'] = i[0].split('_')[2]
    if 'AMAZONIA' in i[0]:
        data_df.loc[i, 'sensor'] = 'AWFI'
    if 'L4_' in i[1]:
        data_df.loc[i, 'bands_num'] = i[1].split('L4_')[1].split('.')[0]
    else:
        data_df.loc[i, 'bands_num'] = i[1].split('L2_')[1].split('.')[0]

# Let's add a more comprehensible definition for bands
bands_dict = {sensor: dict() for sensor in data_df['sensor'].unique()}
bands_dict['PAN10M'] = {'BAND3': 'Red', 'BAND4': 'NIR', 'BAND2': 'Green'}
bands_dict['MUX'] = {'BAND5': 'Blue', 'BAND6': 'Green', 'BAND7': 'Red', 'BAND8': 'NIR'}
bands_dict['WFI'] = {'BAND13': 'Blue', 'BAND14': 'Green', 'BAND15': 'Red', 'BAND16': 'NIR'}
bands_dict['CCD1XS'] = {'BAND1': 'Blue', 'BAND2': 'Green', 'BAND3': 'Red', 'BAND4': 'NIR', 'BAND5': 'PAN'}
bands_dict['AWFI'] = {'BAND1': 'Blue', 'BAND2': 'Green', 'BAND3': 'Red', 'BAND4': 'NIR'}
data_df['bands_name'] = ''
for i, r in data_df.iterrows():
    data_df.loc[i, 'bands_name'] = bands_dict[r.sensor][r.bands_num]
data_df

In [ ]:
for product in data_df.loc[data_df['sensor']!='PAN10M'].index.get_level_values(0).unique():
    # Select the product
    df = data_df.loc[product]
    fsize = 20
    # Prepare the figure
    fig, axs = plt.subplots(1, 2, figsize=(24, 12))
    bands = dict()
    for i, r in df.iterrows():
        # Load the band
        with rasterio.open(i, 'r') as src:
            raster = src.read()
            mask = src.read_masks()
        # Add it to the bands dictionary
        bands[r['bands_name']] = raster.copy()
    
    # "Glue" the bands together
    img = np.concatenate((bands['Red'], bands['Green'], bands['Blue']), axis=0)
    bit_depth = img.dtype
    if img.dtype == np.int16:
        img = (img-img.min())/(img.max()-img.min())
    axs[0].imshow(np.moveaxis(img, 0, 2))
    axs[1].imshow(np.squeeze(mask), cmap='gray')
    fig.suptitle(f"{product} bands and no data mask, {bit_depth} bits depth", fontsize=fsize)
    plt.show()

## Landsat sample

In [ ]:
bands = glob.glob(os.path.join(DATA_ROOT, '**/*.TIF'), recursive=True)
tmp_df = pd.concat([pd.concat({acq.split('/')[4]: pd.DataFrame(index=[acq])}, names=['Original_product', 'Bands path']) for acq in bands])
tmp_df['height'] = 0
tmp_df['height'] = 0
tmp_df['sensor'] = 'OLI'
tmp_df['bands_num'] = ''
tmp_df['bands_name'] = ''
data_df = pd.concat([data_df, tmp_df])
data_df

### Let's add the info we need from the profiles

In [ ]:
# We can add some information in between too
for product in tmp_df.index.get_level_values(0).unique():
    # Select the product
    df = data_df.loc[product]
    fsize = 20
    for idx, band in enumerate(df.index.get_level_values(0).unique()):
        # Load the band
        with rasterio.open(band, 'r') as src:
            # Add info
            data_df.loc[(product, band), 'height'] = src.profile['height']
            data_df.loc[(product, band), 'width'] = src.profile['width']
            data_df.loc[(product, band), 'bit_depth'] = src.profile['dtype']

### Let's plot these bands

In [ ]:
# We can add some information in between too
for product in tmp_df.index.get_level_values(0).unique():
    # Select the product
    df = data_df.loc[product]
    fsize = 20
    # Prepare the figure
    if len(df) > 3:
        fig, axs = plt.subplots(1, len(df), figsize=(16*len(df), 16))
        fsize = 35
    else:
        fig, axs = plt.subplots(1, len(df), figsize=(12*len(df), 12))
    for idx, band in enumerate(df.index.get_level_values(0).unique()):
        # Load the band
        with rasterio.open(band, 'r') as src:
            raster = src.read()
            data_df.loc[(product, band), 'height'] = src.profile['height']
            data_df.loc[(product, band), 'width'] = src.profile['width']
            data_df.loc[(product, band), 'bit_depth'] = raster.dtype
        axs[idx].imshow(np.squeeze(raster), cmap='gray')
        axs[idx].set_title(f"{band.split('/')[-1].split('.')[0]} raster,\ndtype {raster.dtype}, dimensions {raster.shape}", fontsize=fsize)
    fig.suptitle(f"{product} bands", fontsize=fsize)
    plt.show()

### Let's add some sensors information
For the **[OLI](https://www.usgs.gov/faqs/what-are-band-designations-landsat-satellites) sensor**, the bands are:
1. Band 1 = 0.43-0.45 µm (Coastal aerosol band);
2. Band 2 =	0.45-0.51 µm (Blue band);
3. Band 3 = 0.53-0.59 µm (Green band);
4. Band 4 = 0.64-0.67 µm (Red band).

In [ ]:
# Let's pick up the sensors first and bands first
for i, r in tmp_df.iterrows():
    data_df.loc[i, 'bands_num'] = i[1].split('/')[-1].split('_')[1].split('.')[0]

# Let's add a more comprehensible definition for bands
bands_dict = {sensor: dict() for sensor in data_df['sensor'].unique()}
bands_dict['PAN10M'] = {'BAND3': 'Red', 'BAND4': 'NIR', 'BAND2': 'Green'}
bands_dict['MUX'] = {'BAND5': 'Blue', 'BAND6': 'Green', 'BAND7': 'Red', 'BAND8': 'NIR'}
bands_dict['WFI'] = {'BAND13': 'Blue', 'BAND14': 'Green', 'BAND15': 'Red', 'BAND16': 'NIR'}
bands_dict['CCD1XS'] = {'BAND1': 'Blue', 'BAND2': 'Green', 'BAND3': 'Red', 'BAND4': 'NIR', 'BAND5': 'PAN'}
bands_dict['AWFI'] = {'BAND1': 'Blue', 'BAND2': 'Green', 'BAND3': 'Red', 'BAND4': 'NIR'}
bands_dict['OLI'] = {'B1': 'Coastal aerosol', 'B2': 'Blue', 'B3': 'Green', 'B4': 'Red'}
for i in tmp_df.index:
    data_df.loc[i, 'bands_name'] = bands_dict[data_df.loc[i, 'sensor']][data_df.loc[i, 'bands_num']]
data_df

### Let's glue the bands all together and have a look at the full image

In [ ]:
for product in ['LO82240772021365CUB00']:
    # Select the product
    df = data_df.loc[product]
    fsize = 20
    # Prepare the figure
    fig, axs = plt.subplots(1, 2, figsize=(24, 12))
    bands = dict()
    for i, r in df.iterrows():
        # Load the band
        with rasterio.open(i, 'r') as src:
            raster = src.read()
            mask = src.read_masks()
        # Add it to the bands dictionary
        bands[r['bands_name']] = raster.copy()
    
    # "Glue" the bands together
    img = np.concatenate((bands['Red'], bands['Green'], bands['Blue']), axis=0)
    bit_depth = img.dtype
    if img.dtype != np.uint8:
        img = (img-img.min())/(img.max()-img.min())
    axs[0].imshow(np.moveaxis(img, 0, 2))
    axs[1].imshow(np.squeeze(mask), cmap='gray')
    fig.suptitle(f"{product} bands and no data mask, {bit_depth} bits depth", fontsize=fsize)
    plt.show()

### Let's add a uniform scaling just for visualization purposes

In [ ]:
for product in ['LO82240772021365CUB00']:
    # Select the product
    df = data_df.loc[product]
    fsize = 20
    # Prepare the figure
    fig, axs = plt.subplots(1, 2, figsize=(24, 12))
    bands = dict()
    for i, r in df.iterrows():
        # Load the band
        with rasterio.open(i, 'r') as src:
            raster = src.read()
            mask = src.read_masks()
        # Add it to the bands dictionary
        bands[r['bands_name']] = raster.copy()
    
    # "Glue" the bands together
    img = np.concatenate((bands['Red'], bands['Green'], bands['Blue']), axis=0)
    bit_depth = img.dtype
    if img.dtype != np.uint8:
        img = quantile_scaling(img, 'uniform')
    axs[0].imshow(np.moveaxis(img, 0, 2))
    axs[1].imshow(np.squeeze(mask), cmap='gray')
    fig.suptitle(f"{product} bands and no data mask, {bit_depth} bits depth", fontsize=fsize)
    plt.show()

## Sentinel2

In [ ]:
bands = glob.glob(os.path.join(DATA_ROOT, '**/IMG_DATA/R10m/*B0*.jp2'), recursive=True)
tmp_df = pd.concat([pd.concat({acq.split('/')[4]: pd.DataFrame(index=[acq])}, names=['Original_product', 'Bands path']) for acq in bands])
tmp_df['height'] = 0
tmp_df['height'] = 0
tmp_df['sensor'] = 'OLI'
tmp_df['bands_num'] = ''
tmp_df['bands_name'] = ''
data_df = pd.concat([data_df, tmp_df])
data_df

### Let's add the info we need from the profile

In [ ]:
# We can add some information in between too
for product in tmp_df.index.get_level_values(0).unique():
    # Select the product
    df = data_df.loc[product]
    fsize = 20
    for idx, band in enumerate(df.index.get_level_values(0).unique()):
        # Load the band
        with rasterio.open(band, 'r') as src:
            # Add info
            data_df.loc[(product, band), 'height'] = src.profile['height']
            data_df.loc[(product, band), 'width'] = src.profile['width']
            data_df.loc[(product, band), 'bit_depth'] = src.profile['dtype']

### Let's have a look at the bands

In [ ]:
# We can add some information in between too
for product in tmp_df.index.get_level_values(0).unique():
    # Select the product
    df = data_df.loc[product]
    fsize = 20
    # Prepare the figure
    if len(df) > 3:
        fig, axs = plt.subplots(1, len(df), figsize=(16*len(df), 16))
        fsize = 35
    else:
        fig, axs = plt.subplots(1, len(df), figsize=(12*len(df), 12))
    for idx, band in enumerate(df.index.get_level_values(0).unique()):
        # Load the band
        with rasterio.open(band, 'r') as src:
            raster = src.read()
            data_df.loc[(product, band), 'height'] = src.profile['height']
            data_df.loc[(product, band), 'width'] = src.profile['width']
            data_df.loc[(product, band), 'bit_depth'] = raster.dtype
        axs[idx].imshow(np.squeeze(raster), cmap='gray')
        axs[idx].set_title(f"{band.split('/')[-1].split('.')[0]} raster,\ndtype {raster.dtype}, dimensions {raster.shape}", fontsize=fsize)
    fig.suptitle(f"{product} bands", fontsize=fsize)
    plt.show()

### Let's add some sensor information too

In [ ]:
# Let's pick up the sensors first and bands first
for i, r in tmp_df.iterrows():
    data_df.loc[i, 'bands_num'] = i[1].split('/')[-1].split('_')[-2]
    data_df.loc[i, 'sensor'] = i[0].split('_')[0]

# Let's add a more comprehensible definition for bands
bands_dict = {sensor: dict() for sensor in data_df['sensor'].unique()}
bands_dict['PAN10M'] = {'BAND3': 'Red', 'BAND4': 'NIR', 'BAND2': 'Green'}
bands_dict['MUX'] = {'BAND5': 'Blue', 'BAND6': 'Green', 'BAND7': 'Red', 'BAND8': 'NIR'}
bands_dict['WFI'] = {'BAND13': 'Blue', 'BAND14': 'Green', 'BAND15': 'Red', 'BAND16': 'NIR'}
bands_dict['CCD1XS'] = {'BAND1': 'Blue', 'BAND2': 'Green', 'BAND3': 'Red', 'BAND4': 'NIR', 'BAND5': 'PAN'}
bands_dict['AWFI'] = {'BAND1': 'Blue', 'BAND2': 'Green', 'BAND3': 'Red', 'BAND4': 'NIR'}
bands_dict['OLI'] = {'B1': 'Coastal aerosol', 'B2': 'Blue', 'B3': 'Green', 'B4': 'Red'}
bands_dict['S2A'] = {'B02': 'Blue', 'B03': 'Green', 'B04': 'Red', 'B08': 'NIR'}
bands_dict['S2B'] = {'B02': 'Blue', 'B03': 'Green', 'B04': 'Red', 'B08': 'NIR'}
for i in tmp_df.index:
    data_df.loc[i, 'bands_name'] = bands_dict[data_df.loc[i, 'sensor']][data_df.loc[i, 'bands_num']]
data_df

### Let's glue the RGB bands together

In [ ]:
for product in tmp_df.index.get_level_values(0).unique():
    # Select the product
    df = data_df.loc[product]
    fsize = 20
    # Prepare the figure
    fig, axs = plt.subplots(1, 2, figsize=(24, 12))
    bands = dict()
    for i, r in df.iterrows():
        # Load the band
        with rasterio.open(i, 'r') as src:
            raster = src.read()
            mask = src.read_masks()
        # Add it to the bands dictionary
        bands[r['bands_name']] = raster.copy()
    
    # "Glue" the bands together
    img = np.concatenate((bands['Red'], bands['Green'], bands['Blue']), axis=0)
    bit_depth = img.dtype
    if img.dtype != np.uint8:
        img = (img-img.min())/(img.max()-img.min())
    axs[0].imshow(np.moveaxis(img, 0, 2))
    axs[1].imshow(np.squeeze(mask), cmap='gray')
    fig.suptitle(f"{product} bands and no data mask, {bit_depth} bits depth", fontsize=fsize)
    plt.show()

### Let's do a uniform quantization in order to show better the images

In [ ]:
for product in tmp_df.index.get_level_values(0).unique():
    # Select the product
    df = data_df.loc[product]
    fsize = 20
    # Prepare the figure
    fig, axs = plt.subplots(1, 2, figsize=(24, 12))
    bands = dict()
    for i, r in df.iterrows():
        # Load the band
        with rasterio.open(i, 'r') as src:
            raster = src.read()
            mask = src.read_masks()
        # Add it to the bands dictionary
        bands[r['bands_name']] = raster.copy()
    
    # "Glue" the bands together
    img = np.concatenate((bands['Red'], bands['Green'], bands['Blue']), axis=0)
    bit_depth = img.dtype
    if img.dtype != np.uint8:
        img = quantile_scaling(img, 'uniform')
    axs[0].imshow(np.moveaxis(img, 0, 2))
    axs[1].imshow(np.squeeze(mask), cmap='gray')
    fig.suptitle(f"{product} bands and no data mask, {bit_depth} bits depth", fontsize=fsize)
    plt.show()

### Let's add also the images from PlanetScope
We have 3 new products to add:
1. An RGB image from PlanetScope 248b;
2. An RGB image from RapidEye2;
3. An RGB image from RapidEye4.  

All images are the same kinds of products (orthorectified and surface radiance corrected).

In [ ]:
bands = glob.glob(os.path.join(DATA_ROOT, '**/*.tif'), recursive=True)
bands = [band for band in bands if ('analytic' in band) and ('udm.tif' not in band) and ('udm2.tif' not in band)]
tmp_df = pd.concat([pd.concat({acq.split('/')[4]: pd.DataFrame(index=[acq])}, names=['Original_product', 'Bands path']) for acq in bands])
tmp_df['height'] = 0
tmp_df['height'] = 0
tmp_df['sensor'] = 'OLI'
tmp_df['bands_num'] = ''
tmp_df['bands_name'] = ''
tmp_df
data_df = pd.concat([data_df, tmp_df])
data_df

### Let's add height and width

In [ ]:
# We can add some information in between too
for product in tmp_df.index.get_level_values(0).unique():
    # Select the product
    df = data_df.loc[product]
    fsize = 20
    for idx, band in enumerate(df.index.get_level_values(0).unique()):
        # Load the band
        with rasterio.open(band, 'r') as src:
            # Add info
            data_df.loc[(product, band), 'height'] = src.profile['height']
            data_df.loc[(product, band), 'width'] = src.profile['width']
            data_df.loc[(product, band), 'bit_depth'] = src.profile['dtype']
data_df

### Let's have a look at the bands
## Watch out!
In this case we have a unique raster for all bands! The order should be BGRN (N = Near Infrared).

In [ ]:
# We can add some information in between too
for product in tmp_df.index.get_level_values(0).unique():
    # Select the product
    df = data_df.loc[product]
    fsize = 20
    # Prepare the figure
    fig, axs = plt.subplots(1, len(df), figsize=(12*len(df), 12))
    for idx, band in enumerate(df.index.get_level_values(0).unique()):
        # Load the band
        with rasterio.open(band, 'r') as src:
            raster = src.read()
            data_df.loc[(product, band), 'height'] = src.profile['height']
            data_df.loc[(product, band), 'width'] = src.profile['width']
            data_df.loc[(product, band), 'bit_depth'] = raster.dtype
        raster = raster[[2, 1, 0]]
        if raster.dtype != np.uint8:
            raster = quantile_scaling(raster, 'uniform')
        axs.imshow(np.moveaxis(raster, 0, 2))
        axs.set_title(f"{band.split('/')[-1].split('.')[0]} raster,\ndtype {raster.dtype}, dimensions {raster.shape}", fontsize=fsize)
    fig.suptitle(f"{product} bands", fontsize=fsize)
    plt.show()

### We have a lots of useful info in the masks!

In [ ]:
# We can add some information in between too
for product in tmp_df.index.get_level_values(0).unique():
    # Select the product
    df = data_df.loc[product]
    fsize = 20
    # Prepare the figure
    for band in df.index.get_level_values(0).unique():
        # Load the band
        if 'RapidEye' in band:
            band = band.replace('Analytic_SR', 'udm')
            with rasterio.open(band, 'r') as src:
                masks = src.read()
            fig, axs = plt.subplots(1, 1, figsize=(12, 12))
            axs.imshow(np.squeeze(masks), cmap='gray')
            axs.set_title(f"{band.split('/')[-1].split('.')[0]} mask {idx},\ndimensions {masks.shape}", fontsize=fsize)
            fig.suptitle(f"{product} bands", fontsize=fsize)
            plt.show()
        else:
            band = band.replace('BGRN_SR', 'udm2')
            with rasterio.open(band, 'r') as src:
                masks = src.read()
            fig, axs = plt.subplots(1, len(masks), figsize=(12*len(masks), 12))
            for idx, mask in enumerate(masks):
                axs[idx].imshow(np.squeeze(mask), cmap='gray')
                axs[idx].set_title(f"{band.split('/')[-1].split('.')[0]} mask {idx},\ndimensions {raster.shape}", fontsize=fsize)
            fig.suptitle(f"{product} bands", fontsize=fsize)
            plt.show()

### The UDM2 provides more info for all pixels, but also the cloud and shadow pixels are reported as unclear!
We would like to have them instead, can we try to combine the info from both in a single mask?

Let's also extract binary mask from the UDM files of RapidEye

In [ ]:
# We can add some information in between too
for product in tmp_df.index.get_level_values(0).unique():
    # Select the product
    df = data_df.loc[product]
    fsize = 20
    # Prepare the figure
    for band in df.index.get_level_values(0).unique():
        # Load the band
        if 'RapidEye' in band:
            band = band.replace('Analytic_SR', 'udm')
            with rasterio.open(band, 'r') as src:
                masks = src.read()
            fig, axs = plt.subplots(1, 1, figsize=(12, 12))
            masks[masks==2] = 0  # fix the cloud pixels
            masks[masks==0] = 255  # revert the mask
            masks[masks!=255] = 0  # change the other pixels to 0
            axs.imshow(np.squeeze(masks), cmap='gray', clim=[0, 255])
            axs.set_title(f"{band.split('/')[-1].split('.')[0]} mask {idx},\ndimensions {masks.shape}", fontsize=fsize)
            fig.suptitle(f"{product} bands", fontsize=fsize)
            plt.show()
        else:
            band = band.replace('BGRN_SR', 'udm2')
            with rasterio.open(band, 'r') as src:
                masks = src.read()
            fig, axs = plt.subplots(1, 1, figsize=(12, 12))
            axs.imshow(masks[0]+masks[2]+masks[5], cmap='gray')
            axs.set_title(f"{band.split('/')[-1].split('.')[0]} combined mask,\ndimensions {masks.shape}", fontsize=fsize)
            fig.suptitle(f"{product} bands", fontsize=fsize)
            plt.show()

### Let's add some sensor info

In [ ]:
data_df.loc['PlanetScope_Campinas_psorthotile_analytic_sr_udm2', 'sensor'] = 'PSB.SD'
data_df.loc['PlanetScope_Campinas_psorthotile_analytic_sr_udm2', 'bands_name'] = 'RGBN'
data_df.loc['RapidEye2_Santos_reorthotile_analytic_sr', 'sensor'] = 'JSS 56'
data_df.loc['RapidEye2_Santos_reorthotile_analytic_sr', 'bands_name'] = 'RGBReN'
data_df.loc['RapidEye4_Sao_Miguel_Arcangeo_reorthotile_analytic_sr', 'sensor'] = 'JSS 56'
data_df.loc['RapidEye4_Sao_Miguel_Arcangeo_reorthotile_analytic_sr', 'bands_name'] = 'RGBReN'
data_df

### All right! Now we have our training images :)

## Let's plot some statistics shown in the paper

In [ ]:
color_dict = {'Red': 'r', 'Green': 'g', 'Blue': 'b'}
plt.rcParams["font.family"] = "serif"
plt.rcParams["font.serif"] = ["Times New Roman"]
for product in data_df.drop('CBERS_4_PAN10M_20220905_151_116').index.get_level_values(0).unique():
    # Select the product
    df = data_df.loc[product]
    fsize = 30
    # Prepare the figure
    fig, axs = plt.subplots(1, 3, figsize=(27, 9))
    for i, r in df.iterrows():
        if r['bands_name'] in ['Red', 'Blue', 'Green']:
            # Load the band
            with rasterio.open(i, 'r') as src:
                raster = src.read()
                mask = src.read_masks()

            # Plot the standard pixel distribution
            bit_depth = raster.dtype
            dtype_info = np.iinfo(bit_depth)  # we know all images to be integers
            axs[0].hist(raster[mask==255].flatten(), bins=200, range=(0, dtype_info.max), color=color_dict[r['bands_name']])
            axs[0].grid(True)
            axs[0].set_title('Standard dist', fontsize=fsize)
            axs[0].set_xlabel('Pixel value', fontsize=fsize)
            axs[0].set_ylabel('\% of occurrence', fontsize=fsize)
            axs[0].tick_params(axis='both', labelsize=fsize-5)
            # Plot the pixel distribution up to the max value
            max_value = raster[mask==255].max()
            axs[1].hist(raster[mask==255].flatten(), bins=200, range=(0, max_value), color=color_dict[r['bands_name']])
            axs[1].grid(True)
            axs[1].set_title('Max-value dist', fontsize=fsize)
            axs[1].set_xlabel('Pixel value', fontsize=fsize)
            axs[1].set_ylabel('\% of occurrence', fontsize=fsize)
            axs[1].tick_params(axis='both', labelsize=fsize-5)
            # Plot the distribution up to the 99th percentile
            q = np.percentile(raster[mask==255], 99)
            axs[2].hist(raster[mask==255].flatten(), bins=200, range=(0, q), color=color_dict[r['bands_name']])
            axs[2].grid(True)
            axs[2].set_title(r'99-th percentile dist', fontsize=fsize)
            axs[2].set_xlabel('Pixel value', fontsize=fsize)
            axs[2].set_ylabel('\% of occurrence', fontsize=fsize)
            axs[2].tick_params(axis='both', labelsize=fsize-5)
        
        elif r['bands_name'] in ['RGBN', 'RGBReN']:
            
            # Load the band
            with rasterio.open(i, 'r') as src:
                raster = src.read()[[2, 1, 0]]
                
            # Load the mask
            if 'RapidEye' in product:
                with rasterio.open(i.replace('Analytic_SR', 'udm'), 'r') as src:
                    mask = src.read()
                mask[mask==2] = 0  # fix the cloud pixels
                mask[mask==0] = 255  # revert the mask
                mask[mask!=255] = 0  # change the other pixels to 0
            else:
                with rasterio.open(i.replace('BGRN_SR', 'udm2'), 'r') as src:
                    mask = src.read()
                mask = mask[0]+mask[2]+mask[5]
            mask = np.squeeze(mask)
            
            # Plot the standard pixel distribution
            for band_idx, band_name in enumerate(['Red', 'Green', 'Blue']):
                bit_depth = raster.dtype
                dtype_info = np.iinfo(bit_depth)  # we know all images to be integers
                axs[0].hist(raster[band_idx][mask>0].flatten(), bins=200, range=(0, dtype_info.max), color=color_dict[band_name])
                axs[0].grid(True)
                axs[0].set_title('Standard dist', fontsize=fsize)
                axs[0].set_xlabel('Pixel value', fontsize=fsize)
                axs[0].set_ylabel('\% of occurrence', fontsize=fsize)
                axs[0].tick_params(axis='both', labelsize=fsize-5)
                # Plot the pixel distribution up to the max value
                max_value = raster[band_idx][mask>0].max()
                axs[1].hist(raster[band_idx][mask>0].flatten(), bins=200, range=(0, max_value), color=color_dict[band_name])
                axs[1].grid(True)
                axs[1].set_title('Max-value dist', fontsize=fsize)
                axs[1].set_xlabel('Pixel value', fontsize=fsize)
                axs[1].set_ylabel('\% of occurrence', fontsize=fsize)
                axs[1].tick_params(axis='both', labelsize=fsize-5)
                # Plot the distribution up to the 99th percentile
                q = np.percentile(raster[band_idx][mask>0], 99)
                axs[2].hist(raster[band_idx][mask>0].flatten(), bins=200, range=(0, q), color=color_dict[band_name])
                axs[2].grid(True)
                axs[2].set_title(r'99-th percentile dist', fontsize=fsize)
                axs[2].set_xlabel('Pixel value', fontsize=fsize)
                axs[2].set_ylabel('\% of occurrence', fontsize=fsize)
                axs[2].tick_params(axis='both', labelsize=fsize-5)

    # Add the title and show
    fig.suptitle(f"{product} bands and PMF, {bit_depth} bits depth", fontsize=fsize)
    plt.show()